# XGBoost Regression Implememntation

In [1]:
import pandas as pd        
import numpy as np         
import matplotlib.pyplot as plt     
import seaborn as sns  
import plotly.express as px 
import warnings 
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
df=pd.read_csv('cardekho_imputated.csv',index_col=0)
df.head()   

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


## Data Cleaning 

### Handling missing values
1. handling missing values
2. handling duplicates
3. Check datatype
4. Understand the dataset

In [3]:
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [4]:
# car name , brand ,model all same ,so drop 2 of them
# keep only model
df.drop('car_name',axis=1,inplace=True)
df.drop('brand',axis=1,inplace=True)

In [5]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [6]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [7]:
# Get all the numeric features
num_features=[feature for feature in df.columns if df[feature].dtype !='O']
print('Num of Numerical Features : ',len(num_features))

# Get all the categorical features
cat_features=[feature for feature in df.columns if df[feature].dtype =='O']
print('Num of Numerical Features : ',len(cat_features))

# discrete features
discrete_features=[feature for feature in num_features if len(df[feature].unique())<=25]
print('Num of discrete feature :',len(discrete_features))

# continuous features
continuous_features =[feature for feature in num_features if len(df[feature].unique())>=25]
print('Num of continuous feature :',len(continuous_features ))

Num of Numerical Features :  7
Num of Numerical Features :  4
Num of discrete feature : 2
Num of continuous feature : 5


In [8]:
# independent and depemndent features
from sklearn.model_selection import train_test_split

X=df.drop(['selling_price'],axis=1)
y=df['selling_price']


In [9]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


# FEATURE ENCODING AND SCALING
## oneHotEncoding for column having lesser unique values

In [10]:
len(df['model'].unique())
#labelEncoding

120

In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X['model']=le.fit_transform(X['model'])

In [12]:
X['model'].unique()

array([  7,  54, 118,  38, 100, 117,  96,  88,  97,  32,  29,  24,  16,
        60,  13,  89,  95,  30,  25,  14,  45,  64,  10,  84, 114,  63,
        59,  78,  85,  68,  11,  44,  98,  91,  73,  86,  23,  49,   1,
        19,  41,  62,  61,   0,   4,  92,  42,  83,  90,  58,  79,  39,
         5,  36,  74, 116,   2, 111, 106,  57,  26,  31,  22, 103,  77,
        46,  87,  70, 113,  34,  82,  99, 112,  93,  37, 101,  20,  40,
       115,  47,   3,  81,  33,  12, 107,  51,  28,  18,  65,  80,  94,
        56, 104,  71,  27,  17, 119,  53,  67, 105,  35, 109,  43,   6,
        66,  50,  48, 102, 110, 108,  72,   9,   8,  69,  21,  15,  76,
        52,  75,  55])

In [13]:
X

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5
...,...,...,...,...,...,...,...,...,...,...
19537,117,9,10723,Dealer,Petrol,Manual,19.81,1086,68.05,5
19540,42,2,18000,Dealer,Petrol,Manual,17.50,1373,91.10,7
19541,77,6,67000,Dealer,Diesel,Manual,21.14,1498,103.52,5
19542,114,5,3800000,Dealer,Diesel,Manual,16.00,2179,140.00,7


In [14]:
len(df['seller_type'].unique()),len(df['fuel_type'].unique()),len(df['transmission_type'].unique())
#Onehotencoding can be performed

(3, 5, 2)

In [15]:
# create column transformer with 3 types of transformers
num_features=X.select_dtypes(exclude='object').columns
onehot_columns=['seller_type','fuel_type','transmission_type']


from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer 

numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder(drop='first')

preprocessor =ColumnTransformer (
    [
        ("OneHotEncoder",oh_transformer,onehot_columns),
        ("StandardScaler",numeric_transformer,num_features)
    ],remainder='passthrough'
)

In [16]:
X=preprocessor.fit_transform(X)

In [17]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,1.247335,-0.000276,-1.324259,-1.263352,-0.403022
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.225693,-0.343933,-0.690016,-0.192071,-0.554718,-0.432571,-0.403022
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.536377,1.647309,0.084924,-0.647583,-0.554718,-0.479113,-0.403022
3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,-0.360667,0.292211,-0.936610,-0.779312,-0.403022
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.666211,-0.012060,-0.496281,0.735736,0.022918,-0.046502,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.508844,0.983562,-0.869744,0.026096,-0.767733,-0.757204,-0.403022
15407,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.556082,-1.339555,-0.728763,-0.527711,-0.216964,-0.220803,2.073444
15408,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.407551,-0.012060,0.220539,0.344954,0.022918,0.068225,-0.403022
15409,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.426247,-0.343933,72.541850,-0.887326,1.329794,0.917158,2.073444


In [18]:
#train-test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape

((12328, 14), (3083, 14))

In [19]:
X_train

array([[ 0.        ,  0.        ,  1.        , ...,  1.75390551,
         2.66249771, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ..., -0.55087963,
        -0.38602844, -0.40302241],
       [ 0.        ,  0.        ,  1.        , ...,  0.89033072,
         3.27453006, -0.40302241],
       ...,
       [ 1.        ,  0.        ,  0.        , ..., -0.9366097 ,
        -0.78070786, -0.40302241],
       [ 0.        ,  0.        ,  0.        , ..., -0.55471774,
        -0.43582879, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ..., -0.04616815,
         0.06194201, -0.40302241]])

# MODEL TRAINING AND MODEL SELECTION

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [21]:
# FUNCTION TO EVALUATE A MODEL 
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [22]:
# Begining Model Training
models ={
    "Linear Regression":LinearRegression(),
    "lasso":Lasso(),
    "ridge":Ridge(),
    "K-neighbours Regressor":KNeighborsRegressor(),
    "Decision Tree Regressor":DecisionTreeRegressor(),
    "Random Forest" :RandomForestRegressor(),
    "Adaboost Regressor":AdaBoostRegressor(),
    "Gradient Boost":GradientBoostingRegressor(),
    "XGBoost":XGBRegressor()
}

for i in range(len(list(models))) :
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    
    #predictions
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    #evaluate the train and test dataset
    model_train_mae,model_train_rmse,model_train_r2=evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_rmse,model_test_r2=evaluate_model(y_test,y_test_pred)
    

    print(list(models.keys())[i])


    print('Model Performance for training set')

    print('-Root Mean Squared :{: .4f}'.format(model_train_rmse))
    print('-Mean absolute error :{: .4f}'.format(model_train_mae))
    print('-r2 score :{: .4f}'.format(model_train_r2))



    print('------------------------------------------------------------')
    print('Model Performance for test set')

    print('-Root Mean Squared :{: .4f}'.format(model_test_rmse))
    print('-Mean absolute error :{: .4f}'.format(model_test_mae))
    print('-r2 score :{: .4f}'.format(model_test_r2))


    print('='*35)
    print('\n')

Linear Regression
Model Performance for training set
-Root Mean Squared : 553855.6665
-Mean absolute error : 268101.6071
-r2 score : 0.6218
------------------------------------------------------------
Model Performance for test set
-Root Mean Squared : 502543.5930
-Mean absolute error : 279618.5794
-r2 score : 0.6645


lasso
Model Performance for training set
-Root Mean Squared : 553855.6710
-Mean absolute error : 268099.2226
-r2 score : 0.6218
------------------------------------------------------------
Model Performance for test set
-Root Mean Squared : 502542.6696
-Mean absolute error : 279614.7461
-r2 score : 0.6645


ridge
Model Performance for training set
-Root Mean Squared : 553856.3160
-Mean absolute error : 268059.8015
-r2 score : 0.6218
------------------------------------------------------------
Model Performance for test set
-Root Mean Squared : 502533.8230
-Mean absolute error : 279557.2169
-r2 score : 0.6645


K-neighbours Regressor
Model Performance for training set
-Ro

#### Hyperparameter Tuning


In [23]:
## hyperparameter Tuning
KNN_params ={"n_neighbors":[2,3,10,20,40,50]}
rf_params={
    "max_depth":[5,8,15,None,10],
    "max_features":[5,7,'auto',8],
    "min_samples_split":[2,8,15,20],
    "n_estimators":[100,200,500,1000]
}
xgboost_params={
    "learning_rate":[0.1,0.01],
    "max_depth":[5,8,12,20,30],
    "n_estimators":[100,200,300],
    'colsample_bytree':[0.5,0.8,1,0.3,0.4]
}

In [24]:
#model list for hyperparameter tuning
randomcv_models=[
    ("RF",RandomForestRegressor(),rf_params),
    ("KNN",KNeighborsRegressor(),KNN_params),
    ("XGB",XGBRegressor(),xgboost_params)

]

In [25]:
from sklearn.model_selection import RandomizedSearchCV 
model_param={}

for name,model,params in randomcv_models:
    random=RandomizedSearchCV(estimator=model,param_distributions=params,n_iter=100,cv=3,verbose=2,n_jobs=-1)
    random.fit(X_train,y_train)
    model_param[name]=random.best_params_
    
for model_name in model_param:
    print(f"-------------------Best param for {model_name}-----------------")
    print(model_param[model_name])

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
-------------------Best param for RF-----------------
{'n_estimators': 200, 'min_samples_split': 2, 'max_features': 7, 'max_depth': 15}
-------------------Best param for KNN-----------------
{'n_neighbors': 10}
-------------------Best param for XGB-----------------
{'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.1, 'colsample_bytree': 0.5}


In [26]:
# retraining with best params what we got
models={
    "Random Forest":RandomForestRegressor(n_estimators=200,min_samples_split=2,max_features=7,max_depth=15),
    "K-Neighbors Regressor":KNeighborsRegressor(n_neighbors=10,n_jobs=-1),
    "XG Boost":XGBRegressor(n_estimators=200,max_depth=5,learning_rate=0.1,colsample_bytree=0.5)
}

for i in range(len(list(models))) :
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    
    #predictions
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    #evaluate the train and test dataset
    model_train_mae,model_train_rmse,model_train_r2=evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_rmse,model_test_r2=evaluate_model(y_test,y_test_pred)
    

    print(list(models.keys())[i])


    print('Model Performance for training set')

    print('-Root Mean Squared :{: .4f}'.format(model_train_rmse))
    print('-Mean absolute error :{: .4f}'.format(model_train_mae))
    print('-r2 score :{: .4f}'.format(model_train_r2))



    print('------------------------------------------------------------')
    print('Model Performance for test set')

    print('-Root Mean Squared :{: .4f}'.format(model_test_rmse))
    print('-Mean absolute error :{: .4f}'.format(model_test_mae))
    print('-r2 score :{: .4f}'.format(model_test_r2))


    print('='*35)
    print('\n')

Random Forest
Model Performance for training set
-Root Mean Squared : 130348.6257
-Mean absolute error : 54609.3271
-r2 score : 0.9791
------------------------------------------------------------
Model Performance for test set
-Root Mean Squared : 211377.8494
-Mean absolute error : 97316.3430
-r2 score : 0.9406


K-Neighbors Regressor
Model Performance for training set
-Root Mean Squared : 363460.7706
-Mean absolute error : 103472.0474
-r2 score : 0.8371
------------------------------------------------------------
Model Performance for test set
-Root Mean Squared : 263888.0623
-Mean absolute error : 117496.2131
-r2 score : 0.9075


XG Boost
Model Performance for training set
-Root Mean Squared : 127253.7959
-Mean absolute error : 79408.8263
-r2 score : 0.9800
------------------------------------------------------------
Model Performance for test set
-Root Mean Squared : 332411.3560
-Mean absolute error : 106498.6549
-r2 score : 0.8532


